In [1]:
import requests

def vis_alle_holdeplasser():
 holdeplasser = []
 raw_response = requests.get('https://oslobysykkel.no/api/v1/stations', headers={'Client-Identifier': '15111a56c5b41b49f82d98a9595be3c3'})
 response = raw_response.json()
 for station in response["stations"]:
    holdeplasser.append(station["title"])
 return holdeplasser

def holdeplass_id(navn):
 raw_response = requests.get('https://oslobysykkel.no/api/v1/stations', headers={'Client-Identifier': '15111a56c5b41b49f82d98a9595be3c3'})
 response = raw_response.json()
 for station in response["stations"]:
    if(station["title"]==navn):
        return station["id"]
    
def antall_ledig_sykler(stopp):
 id = holdeplass_id(stopp)
 raw_response = requests.get('https://oslobysykkel.no/api/v1/stations/availability/', headers={'Client-Identifier': '15111a56c5b41b49f82d98a9595be3c3'})
 response = raw_response.json()
 #print(response)
 for station in response["stations"]:
    if(station["id"]==id):
        return station["availability"]["bikes"]

In [21]:
antall_ledig_sykler("Advokat Dehlis Plass")

In [22]:
vis_alle_holdeplasser()

['Nylandsveien',
 'Bentsebrugata',
 'Hans Nielsen Hauges plass',
 'Sjøsiden vest',
 'St. Hanshaugen park vest',
 'Arkitekt Rivertz Plass',
 'Vaterlandsparken',
 'Vår Frelsers gravlund sør',
 'Bjølsen Studentby',
 'Pilestredet Studenthus',
 'Sirkustomta',
 'Bogstadveien',
 'Slemdalsveien',
 'John Colletts plass',
 'Legevakten',
 'Grønlands torg',
 'Stensgata',
 'Tøyenparken',
 'Colosseum Kino',
 'Carl Berners plass sør',
 'Lørenveien',
 'Kirkeristen',
 'Sagene bussholdeplass',
 'Storo Storsenter',
 'HiOA',
 'Lodalen busstopp',
 'Majorstuen',
 'Parkveien',
 'Jakob kirke',
 'Spikersuppa Øst',
 'Oslo Handelsgymnasium',
 'Oslo Hospital',
 'Saga Kino',
 'Ringnes Park',
 'Uelands gate',
 'Hallings gate',
 'Studenterlunden',
 'Tøyen skole',
 'Munkegata trikkestopp',
 'Uranienborgparken',
 'Paléhaven',
 'Norsk Folkemuseum',
 'St. Hanshaugen',
 'Botanisk Hage vest',
 'Hammersborg Torg',
 'Frognerbadet',
 'Birkelunden',
 'Hasle Torg',
 'Torshovparken vest',
 'Schous plass',
 'Bjølsendumpa',
 'Ada

In [26]:
# Speech to text
import speech_recognition as sr 
r = sr.Recognizer()
mic = sr.Microphone()
sr.Microphone.list_microphone_names()

# Text to speech
from gtts import gTTS
from io import BytesIO

# For playing mp3 through the terminal
import os

In [27]:
print("Mic start")
print("Snakk nå")
with mic as source:
    audio = r.listen(source)
print("Mic done")

# Speech to string
tale = r.recognize_google(audio, language='no')
ledig_sykler = antall_ledig_sykler(tale)
if ledig_sykler == 0:
    ledig_sykler = "ingen"

tekst = str(ledig_sykler) + " ledige sykler på " + tale 

print(tekst)

# String to mp3
mp3_fp = BytesIO()
tts = gTTS(tekst, 'no')
tts.save('lyd.mp3')

file = "lyd.mp3"
os.system("afplay " + file)

'''
For linux
# apt install mpg123
file = "file.mp3"
os.system("mpg123 " + file)
'''

Mic start
Snakk nå
Mic done
ingen ledige sykler på Badebakken


'\nFor linux\n# apt install mpg123\nfile = "file.mp3"\nos.system("mpg123 " + file)\n'